In [ ]:
from pathlib import Path
import hjson
import json

import numpy as np
import pandas as pd

# Paths definition
src_path = Path('/home/patomareao/Nextcloud/Juanmi_MED_PSA/WASCOP/Optimización/202312_PID2024')
results_path: Path = src_path / 'results'
data_path: Path = src_path / 'data'

filename_opt_result = '20231218_optimization_results.json'
filename_process_data = '20231218_process_timeseries.csv'

sample_rate = '60s'

# Pre-processing

In [ ]:
# Load system information and data
with open(data_path / 'config.json') as f:
    config = json.load(f)
    
with open(results_path / filename_opt_result) as f:
    opt_results = json.load(f)
        
with open(src_path / "visualizations" / "plot_config.hjson") as f:
    plt_config = hjson.load(f)
    
# Read data from csv, the index column is the one named "time", which is not the first one
df = pd.read_csv(results_path / filename_process_data, parse_dates=True, index_col='time')

# Set UTC timezone
df = df.tz_localize('UTC')

In [ ]:
# Remove the last 19 min since the system was not operating
df = df[:-19*60]

# Sample every 60 seconds to reduce the size of the dataframe
df = df.resample(sample_rate).mean()

# Combine the optimization results with the timeseries data
# From optimization result, build a dataframe with the same index as df, by concatenating the values using the time field

# First convert the time field to datetime
temp_df = pd.DataFrame(opt_results)
temp_df['time'] = pd.to_datetime(temp_df['time'])
temp_df.set_index('time', inplace=True)
temp_df = temp_df.tz_localize('UTC')

continuous_time_index = pd.date_range(start=temp_df.index.min(), end=temp_df.index.max(), freq='1S')
df_opt = temp_df.reindex(continuous_time_index)
df_opt.ffill(inplace=True)
df_opt = df_opt.resample(sample_rate).mean()

# Add suffix to colums and join with df
df_opt = df_opt.add_suffix('_opt')


df = df.join(df_opt, how='outer')

# Return to original index to be used later in the plot
df_opt = temp_df

In [ ]:
# Filter out unreasonable consumption values

df['Ce_opt'].where(df['Ce_opt'] < 100, np.nan, inplace=True)
df['Cw_opt'].where(df['Cw_opt'] < 1000, np.nan, inplace=True)
df_opt['Ce'].where(df_opt['Ce'] < 100, np.nan, inplace=True)
df_opt['Cw'].where(df_opt['Cw'] < 1000, np.nan, inplace=True)

# Calculate active states for DC and WCT, this should be read from the timeseries data
df['dc_active'] = df['w_dc'] > 11
df['wct_active'] = df['w_wct'] > 21

In [ ]:
# Calculate additional variables

from solhycool_visualizations.calculations import power_consumption

# df["C_e"] = consumption_fit(df["pump flow or freq"]) ...
# df["C"] = df["C_e"] * lambda_e + df["C_w"] * lambda_w

df["Ce_dc"] = power_consumption(df["w_dc"].to_numpy(), actuator='fan_dc')
df["Ce_wct"] = power_consumption(df["w_wct"].to_numpy(), actuator='fan_wct')
df["Ce_c"] = power_consumption(df["q_c"].to_numpy(), actuator='pump_c')

df["Ce"] = df["Ce_dc"] + df["Ce_wct"] + df["Ce_c"] 

# Visualizations

## Frente de pareto
Frentes de pareto acumulados hasta ahora para los distintos casos de estudio, punto seleccionado se muestra resaltado

In [ ]:
from solhycool_visualizations.pareto import pareto_plot

fig = pareto_plot()

fig.show()

## Evolución temporal de consumos
Comparación de los consumos de consumos entre la evolución de la serie temporal y los estimados por el modelo 

In [ ]:
from solhycool_visualizations import consumptions_plot

fig = consumptions_plot()

fig.show()

## Resultados experimentales
Gráfica del sistema completo mostrando comparativa entre las series temporales obtenidas experimentalmente y las estimadas por el modelo

In [ ]:
from solhycool_visualizations.experimental_results import experimental_results_plot

fig = experimental_results_plot()

fig.show()

## Diagrama de la instalación
Diagrama de la instalación para visualizar el punto de operación estimado por el modelo

In [ ]:
from solhycool_visualizations.facility_diagram import generate_facility_diagram
from IPython.display import SVG

fig = generate_facility_diagram()

SVG(etree.tostring(diagram_file))